---
layout: code-post
title: NLP Basic Processing
description: This is just me playing around with basic NLP packages.
tags: [misc]
---

Though I'm interested in NLP, I've never actually played around with NLP models
or any of the packages and basic techniques that NLP relies on. So here I'm
just going to play a bit with spaCy and some a basic algorithm or two.

Outline:
- spaCy
- nltk
- BPE and wordpiece

## spaCy

For my introduction to spaCy, I'm pretty much just running through the 
[spaCy introduction](https://spacy.io/usage/spacy-101). Truly there is
not much reason to read this section, the actualy spaCy documentation 
is much more interesting and likely to be correct! I will point out
that I quickly ran into one of the limitations of using their pretrained
models without fine-tuning them.

In [1]:
import spacy
nlp = spacy.load("fr_core_news_md")

let's see what we can do with a corpus made from the first chapter of Camus' "La peste".
I physically copied the text from [here](https://www.ebooksgratuits.com/html/camus_la_peste.html#_Toc284769571),
though I could've with some difficulty used `beautifulsoup`.

In [13]:
with open("data/la_peste_1.txt", "r") as f:
    text = f.read().replace("\n", " ")
    
doc = nlp(text)

In [3]:
doc[:100]

Les curieux événements qui font le sujet de cette chronique se sont produits en 194., à Oran. De l’avis général, ils n’y étaient pas à leur place, sortant un peu de l’ordinaire. À première vue, Oran est, en effet, une ville ordinaire et rien de plus qu’une préfecture française de la côte algérienne.    La cité elle-même, on doit l’avouer, est laide. D’aspect tranquille, il faut quelque temps pour apercevoir ce qui la rend différente de tant d’autres villes

spaCy does a lot of things including part of speech tagging and lemmatization:

In [4]:
for i in range(10):
    print(doc[i].text, doc[i].pos_, doc[i].lemma_)

Les DET le
curieux ADJ curieux
événements NOUN événement
qui PRON qui
font VERB faire
le DET le
sujet NOUN sujet
de ADP de
cette DET ce
chronique NOUN chronique


We can get named entities by called the `.ents` property of a document. The list of entity labels can be found [here](https://spacy.io/api/annotation#named-entities).

In [5]:
for entity in doc[:2000].ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Arrivé - LOC - Non-GPE locations, mountain ranges, bodies of water
Bernard Rieux - PER - Named person or family.
M. Michel - PER - Named person or family.
M. Michel - PER - Named person or family.
Bernard Rieux - PER - Named person or family.
Rieux - LOC - Non-GPE locations, mountain ranges, bodies of water
Dors si tu peux - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


So that last example shows that the named entities are not perfect. "Dors si tu peux" is just a 
normal phrase. A named entity will span multiple tokens, it's not a special tag of a single token:

In [6]:
doc[1524:1526]

Le matin

Also, the results are very different if you lowercase the results first:

In [7]:
with open("data/la_peste_1.txt", "r") as f:
    text_lower = f.read().replace("\n", " ").lower()
    
doc_lower = nlp(text_lower)

In [8]:
for entity in doc_lower[:2000].ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

oran - LOC - Non-GPE locations, mountain ranges, bodies of water
oran - LOC - Non-GPE locations, mountain ranges, bodies of water
laide - PER - Named person or family.
le soleil incendie - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art
les beaux jours - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art
boulevard - PER - Named person or family.
boulomanes - LOC - Non-GPE locations, mountain ranges, bodies of water
amicales - LOC - Non-GPE locations, mountain ranges, bodies of water
oran - PER - Named person or family.
oran - LOC - Non-GPE locations, mountain ranges, bodies of water
oran - LOC - Non-GPE locations, mountain ranges, bodies of water
bernard - PER - Named person or family.
m. michel - PER - Named person or family.
m. michel - PER - Named person or family.
bernard rieux - PER - Named person or family.
dors si tu peux - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art
au t

If one is going to rely on named entities, then it appears that lowercasing is a hindrance, which makes
sense since in French, as in English, uppercasing is used to denote proper names. Let's go back to the
upperacased version but use the full model and see if its predictions about named entities do better.

In [10]:
nlp_large = spacy.load("fr_core_news_lg")

In [14]:
doc_large = nlp_large(text)

In [15]:
for entity in doc_large[:2000].ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Oran - LOC - Non-GPE locations, mountain ranges, bodies of water
Bernard Rieux - PER - Named person or family.
M. Michel - PER - Named person or family.
M. Michel - PER - Named person or family.
Bernard Rieux - PER - Named person or family.
Rieux - PER - Named person or family.
Dors si tu peux - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Still has "Dors si tu peux" (sleep if you can) but it did get rid of Arrivé which is a past participle.
If we look up the line containing the incorrectly labelled phrase, we find the following:

In [22]:
doc_large[1956:1964]

Dors si tu peux, dit-il.

Which shows that this is clearly a phrase ("Sleep if you can, he said"). Apparently it is possible to train from scratch or fine-tune the models that spaCy uses to predict all these labels and is described in the [documentation](https://spacy.io/usage/training).

Let's call the visualizer, since that's something that spaCy has built in apparently.

In [51]:
from spacy import displacy

displacy.render(nlp("Dors si tu peux, dit-il."), style="dep")

The issue seems to be that it is not recognizing "dors" as the imperative of "dormir." If
we change to the "vous" form from the "tu" form we get the following:

In [52]:
displacy.render(nlp("Dormez si tu peux, dit-il"), style="dep")

Although if we go to the entity visualizer it still does not understand.

In [58]:
displacy.render(nlp("Dors si tu peux, dit-il"), style="ent")
displacy.render(nlp("Dormez si tu peux, dit-il"), style="ent")

And these are the two things that displaCy can do. Does make for some pretty output!

The French models come with language vectors (probably produced by word2vec?) that we can see if we wanted by calling `token.vector` where `token` is a token from a  `doc` object. However, being simply a length 300 numpy array, it's not really interesting to look at\. The tokens can also get similarity scores between its vector and another word's vector.

In [69]:
small_doc = nlp("roi reine renard")
print("Similarity between roi and reine:", small_doc[0].similarity(small_doc[1]))
print("Similarity between roi and renard:", small_doc[0].similarity(small_doc[2]))

Similarity between roi and reine: 0.6281081
Similarity between roi and renard: 0.2194891


## NLTK

From what I can surmise from googling, spaCy and NLTK take different approaches. An app developer is probably going to use spaCy as it provides out of the box ready algorithms for the problems at hand with not many choices. NLTK is more useful to researchers who are trying to build from the ground up, but it's trickier to adapt to well worn problems.

After installing nltk with conda, you can't just immediately call functions such as `sent_tokenize` as you will be
presented with a `LookupError` saying it can't find some resource that it is looking for.
This is because nltk is much more customizable and must be customized before running. Data installation instructions
can be found [here](https://www.nltk.org/data.html). I'll try to keep track of what I downloaded as I go. To
download via GUI, run `nltk.download()` which will open up an interface in a new window and you an download
everything or pieces. On my Mac it downloaded to `~/nltk_data`.

For sentence level tokenization, we rely on punkt, which I installed from an interactive python shell I opened in the
terminal via `nltk.download('punkt)`.

In [74]:
import nltk
tokenizer = nltk.data.load('tokenizers/punkt/PY3/french.pickle')

In [76]:
sentences = tokenizer.tokenize(text)

In [82]:
for i in range(4):
    print(sentences[i])

Les curieux événements qui font le sujet de cette chronique se sont produits en 194., à Oran.
De l’avis général, ils n’y étaient pas à leur place, sortant un peu de l’ordinaire.
À première vue, Oran est, en effet, une ville ordinaire et rien de plus qu’une préfecture française de la côte algérienne.
La cité elle-même, on doit l’avouer, est laide.


In [80]:
sentences[1]

'De l’avis général, ils n’y étaient pas à leur place, sortant un peu de l’ordinaire.'

Apparently punkt is just a sentence level tokenizer. But there are many options! Here's one that
apparently was implemented / invented by someone from my alma mater of OSU. It seems to just be
based on regexes from the [documentation](https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.toktok).
There was another regex based tokenizer called the Moses Tokenizer, but it was removed due to an
incompatible license. It still comes up when you search on stack overflow (via google!).

In [103]:
from nltk.tokenize.toktok import ToktokTokenizer
toktok = ToktokTokenizer()
print(toktok.tokenize(sentences[0])[:8])

['Les', 'curieux', 'événements', 'qui', 'font', 'le', 'sujet', 'de']


Lemmatization seems much harder to find in nltk. There is the `WordNetLemmatizer` that exists in
`nltk.stem`, but it is English only and even then the results seem dubious (based on googling).
It is eassier to find Stemmers, as the snowball stemmer has a French version, but these
do not really work in the desired way. One can download an external lemmatizer
such as the [FrenchLefffLematizer](https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer) which
already exists in the spaCy universe via [spacy-leff](https://spacy.io/universe/project/spacy-lefff).
Here's an example of the French snowball stemmer.

In [108]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("french")
words = toktok.tokenize(sentences[0])[:8]
for word in words:
    print(word, "-", stemmer.stem(word))

Les - le
curieux - curieux
événements - éven
qui - qui
font - font
le - le
sujet - sujet
de - de


So stemming is obviously not great! This makes sense since it's based on characters and removing them and not really grammatical
except where the grammar is reflected very simply at the character level. Even a semi-inaccurate lemmatizer
would probably be better in a lot of cases, I would think, now that lemmatizers are probably decently
cheap to train.

It also seems that NLTK does not have a built in way to handle Named Entity Recognition for French, although
it does for English.

We didn't deal with stopwords in spaCy, though we could have. Let's do it in nltk, showing again how it is language specific as well.
This required to `nltk.download('stopwords')`. Then the stopwords appear to just be a list that onee can compare against
using, e.g., list comprehension.

In [88]:
from nltk.corpus import stopwords

stopwords.words('french')[0:10]

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle']

I think it's clear already that I'll probably be using spaCy if I want to work with French text.
Given the different goals of the projects, it's understandable why ntlk makes the choices it does,
it probably won't suit me as often. Or, perhaps more likley, everything will be custom, which
seems to be likely with language data.

## BPE and WordPiece

Above we've been working at the sentence and word level, but the very large transformers that are currently en vogue are 
based on something more akin to the character level. It's not quite accurate, as the GPT family of models rely
on Byte Pair Encodings (BPE) while BERT uses WordPiece, both of which we try to explain shortly.

### BPE

BPE is relatively simple to explain. Take a corpus and break it into distince words, recording
their frequencies. Split each word into characters while also appending a stop word symbol to
the last letter. At the beginning, each letter or letter+stop is considered as a byte.
Iterate over the text doing the following. Look for the most frequent pair of consecutive
bytes and combine them into a new byte. Do this until you are satisfied that the bytes
are appropriately descriptive. Letting the tokenizer continue forever results in each
word being its own byte, so BPE interpolates between letters and words. However, as stated
previously, orthography and grammar are only somewhat related, so BPE does have some drawbacks.


### WordPiece

WordPiece is a generalization of BPE. The difference lies in how you choose tokens to merge.
At each step, you must have a model trained on the data. The chosen byte merge is the one
which most improves the model. For example, say the model is a trigram which predicts the
next byte from the preovious two bytes. Then you would merge the two bytes which most
improve the model upon merging (once its retrained, I believe). This could clearly be
much more computationally intensive depending on the model. Using WordPiece with a digram
recovers BPE, I think.